In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
root_dir='drive/My Drive/driverdrowsiness_csv/'

In [1]:
# #1->1,2,3,6
# #2->1,3,4,5,6
# #3->1,3,4,5,6
# #4->2,3,4,5
# #5->3,4,5,6

# f1=[1,2,3,4]
# f2=[1,3,4,5,6]
# f3=[1,2,3,4,5,6]

final=pd.DataFrame(columns = ["Participant", "EAR", "MAR", "Circularity", "MOE"])
for fold in [1,2,3,4,5,6]:
  #if fold==1:
    for user in [1,2,3,4,5]:
      for label in [0,10]:
        df1=pd.read_csv(root_dir+'F'+str(fold)+'_U'+str(user)+'_L'+str(label)+'_features.csv')
        df2=pd.read_csv(root_dir+'F'+str(fold)+'_U'+str(user)+'_L'+str(label)+'_labels.csv')
        df1 = pd.concat([df1, df2], axis=1)
        for row in df1['0']:
          row[0]=(fold-1)*6+row[0]
        frames=[final,df1]
        final=pd.concat(frames)

#   if fold==1:
#     for user in f1:
#       for label in [0,10]:
#         df1=pd.read_csv(root_dir+'F'+str(fold)+'_U'+str(user)+'_L'+str(label)+'_features.csv')
#         df2=pd.read_csv(root_dir+'F'+str(fold)+'_U'+str(user)+'_L'+str(label)+'_labels.csv')
#         df1 = df1.set_axis(["Pet", "Color"], axis=1)
#         df1 = pd.concat([df1, df2], axis=1)
#         for row in df1['0']:
#           row[0]=(fold-1)*6+row[0]
#         frames=[final,df1]
#         final=pd.concat(frames)

#   if fold==1:
#     for user in f1:
#       for label in [0,10]:
#         df1=pd.read_csv(root_dir+'F'+str(fold)+'_U'+str(user)+'_L'+str(label)+'_features.csv')
#         df2=pd.read_csv(root_dir+'F'+str(fold)+'_U'+str(user)+'_L'+str(label)+'_labels.csv')
#         df1 = pd.concat([df1, df2], axis=1)
#         for row in df1['0']:
#           row[0]=(fold-1)*6+row[0]
#         frames=[final,df1]
#         final=pd.concat(frames)

In [ ]:
import glob
df_features = [pd.read_csv(f, header=None, names=["Participant", "EAR", "MAR", "Circularity", "MOE"]) for f in glob.glob(root_dir+'F*_U*_L*_features.csv')]
df_labels = [pd.read_csv(f, header=None, names=["Y"]) for f in glob.glob(root_dir+'F*_U*_L*_labels.csv')]

df = pd.concat([pd.concat(df_features,ignore_index=True), pd.concat(df_labels,ignore_index=True)], axis=1)
print(df.shape) 
df.drop(axis=0, index=0,inplace=True,)

# Save merged file
df.to_csv(r'drive/My Drive/driverdrowsiness_csv/final3.csv',index=False)
df.head()

(10604, 6)


,Participant,EAR,MAR,Circularity,MOE,Y
1,1.0,0.291891,0.914267,0.482719,3.132220,10.0
2,1.0,0.298348,0.907725,0.496882,3.042501,10.0
3,1.0,0.293436,0.927617,0.454084,3.161228,10.0
4,1.0,0.307491,0.932127,0.463758,3.031399,10.0
5,1.0,0.333347,0.914072,0.491748,2.742100,10.0


In [ ]:
df_total=pd.read_csv(root_dir+'final3.csv')
cols = df_total.columns.tolist()
cols = cols[-1:] + cols[4:5] + cols[:4]
df_total = df_total[cols]
df_total

,Y,MOE,Participant,EAR,MAR,Circularity
0,10,3.132220,1,0.291891,0.914267,0.482719
1,10,3.042501,1,0.298348,0.907725,0.496882
2,10,3.161228,1,0.293436,0.927617,0.454084
3,10,3.031399,1,0.307491,0.932127,0.463758
4,10,2.742100,1,0.333347,0.914072,0.491748
...,...,...,...,...,...,...
10555,10,2.651222,22,0.300739,0.797325,0.436028
10556,10,2.524995,22,0.313246,0.790945,0.465389
10557,10,2.917181,22,0.279668,0.815843,0.430749
10558,10,2.674906,22,0.300905,0.804894,0.453738


In [ ]:
# t = df_total.drop_duplicates("Participant")
# df_total = df_total[~df_total.Participant.isin(t[t.Y > 0].Participant)]
# df_total.shape

(2832, 6)

In [ ]:
df_alert = df_total[df_total["Y"]==0]

In [ ]:
print(df_alert.shape)

(5280, 6)


In [ ]:
df_alert_1 = df_alert.iloc[0::240, :]
df_alert_2 = df_alert.iloc[1::240, :]
df_alert_3 = df_alert.iloc[2::240, :]

In [ ]:
#Merging them into one dataframe
alert_first3 = [df_alert_1,df_alert_2,df_alert_3]
df_alert_first3 = pd.concat(alert_first3)
df_alert_first3 = df_alert_first3.sort_index()

In [ ]:
#Based on the first 3 "Alert" frames, calculating respondent-wise mean and std for each feature
df_means = df_alert_first3.groupby("Participant")["EAR","MAR","Circularity","MOE"].mean()
df_std = df_alert_first3.groupby("Participant")["EAR","MAR","Circularity","MOE"].std()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(df_means)
print(df_std)

                  EAR       MAR  Circularity       MOE
Participant                                           
1            0.263581  0.970955     0.395454  3.689084
2            0.294268  1.039154     0.441576  3.547873
3            0.376711  1.191817     0.519738  3.164180
4            0.353838  1.228749     0.510286  3.486326
5            0.329513  0.865760     0.448881  2.634166
7            0.235807  1.044421     0.393164  4.484137
8            0.345302  0.977309     0.469180  2.832706
9            0.300963  1.288917     0.455608  4.301215
10           0.191879  1.060966     0.326513  6.003048
11           0.192115  1.026641     0.383478  5.434818
12           0.303139  0.930245     0.433082  3.072348
13           0.299328  0.930767     0.467749  3.119007
14           0.287642  1.042579     0.429302  3.664940
15           0.342567  1.033504     0.506191  3.031356
16           0.221074  0.980338     0.365000  4.435155
17           0.227335  1.161344     0.374484  5.316941
18        

In [ ]:
#Functions for mean and std for each feature
def mean_EAR(Participant):
    return df_means.loc[Participant]["EAR"]

def mean_MAR(Participant):
    return df_means.loc[Participant]["MAR"]

def mean_Circularity(Participant):
    return df_means.loc[Participant]["Circularity"]

def mean_MOE(Participant):
    return df_means.loc[Participant]["MOE"]


def std_EAR(Participant):
    return df_std.loc[Participant]["EAR"]

def std_MAR(respondent):
    return df_std.loc[respondent]["MAR"]

def std_Circularity(respondent):
    return df_std.loc[respondent]["Circularity"]

def std_MOE(respondent):
    return df_std.loc[respondent]["MOE"]

In [ ]:
#Adding respondent-wise mean and std for each feature to each row in the original dataframe
df_total["EAR_mean"] = df_total["Participant"].apply(mean_EAR)
df_total["MAR_mean"] = df_total["Participant"].apply(mean_MAR)
df_total["Circularity_mean"] = df_total["Participant"].apply(mean_Circularity)
df_total["MOE_mean"] = df_total["Participant"].apply(mean_MOE)

df_total["EAR_std"] = df_total["Participant"].apply(std_EAR)
df_total["MAR_std"] = df_total["Participant"].apply(std_MAR)
df_total["Circularity_std"] = df_total["Participant"].apply(std_Circularity)
df_total["MOE_std"] = df_total["Participant"].apply(std_MOE)

KeyError: ignored

In [ ]:
#Calculating normalised features for each row in the original dataframe
df_total["EAR_N"] = (df_total["EAR"]-df_total["EAR_mean"])/ df_total["EAR_std"]
df_total["MAR_N"] = (df_total["MAR"]-df_total["MAR_mean"])/ df_total["MAR_std"]
df_total["Circularity_N"] = (df_total["Circularity"]-df_total["Circularity_mean"])/ df_total["Circularity_std"]
df_total["MOE_N"] = (df_total["MOE"]-df_total["MOE_mean"])/ df_total["MOE_std"]

In [ ]:
#Saving the file to a CSV with all the information
df_total.to_csv('totalwithallinfo.csv',index=False)

In [ ]:
#Saving the file to a CSV with all the information
df_main = df_total.drop(["EAR_mean","MAR_mean","Circularity_mean","MOE_mean","EAR_std","MAR_std","Circularity_std","MOE_std"],axis=1)
df_main.to_csv('totalwithmaininfo.csv',index=False)

In [ ]:
# df_total.to_csv(r'data_drowsiness\prepared\totalwithallinfo.csv',index=False)
# #Saving the file to a CSV with all the information
# df_main = df_total.drop(["EAR_mean","MAR_mean", "Circularity_mean", "MOE_mean", "EAR_std", "MAR_std", "Circularity_std", "MOE_std"], axis=1)
# df_main.to_csv(r'data_drowsiness\prepared\totalwithmaininfo.csv',index=False)